## Libraries

In [1]:
import pandas as pd

## Functions

In [2]:
def read_csv(file_name: str) -> pd.DataFrame:
    path = './data/' + file_name + '.csv'

    data = pd.read_csv(path, sep=',')

    return data

### Question 1

In [3]:
SEGMENT = ['Gold', 'Silver', 'Regular']

def q1(data: pd.DataFrame) -> dict:
    d = {
        'regular': data[data['segment'] == SEGMENT[2]].size,
        'silver': data[data['segment'] == SEGMENT[1]].size,
        'gold': data[data['segment'] == SEGMENT[0]].size
    }

    return d


### Question 2

In [4]:
def q2(data: pd.DataFrame):
    cities = {}

    for city in data['province_city']:
        if (cities.get(city) == None):
            cities[city] = 1
        else:
            cities.update({city: cities.get(city)+1})

    return cities

### Question 3

In [57]:
def q3(data: pd.DataFrame, seg_idx):
    gold_data = data[data['segment'] == SEGMENT[seg_idx]]
    gold_data = gold_data.drop('customer_id', axis=1)

    seg_quant = {}

    for city in gold_data['province_city']:
        if (seg_quant.get(city) == None):
            seg_quant[city] = 1
        else:
            seg_quant.update({city: seg_quant.get(city)+1})

    return seg_quant

### Question 4

In [44]:
def q4(aum: pd.DataFrame, cust: pd.DataFrame):
    def get_city(cust_id: int):
        try:
            row = cust.query('customer_id == ' + str(cust_id))            
            return row.iloc[0]['province_city']
        except:
            return 'Not Found!'
        
    properties = {}

    for _, row in aum.iterrows():
        cust_city = get_city(row['customer_id'])

        if (cust_city == 'Not Found!'):
            continue
        else:
            if (properties.get(cust_city) == None):
                properties[cust_city] = row['amount']
            else:
                properties.update({cust_city: properties.get(cust_city)+row['amount']})

    return properties


### Question 5

In [55]:
def q5(aum: pd.DataFrame, cust: pd.DataFrame):
    def get_segment(cust_id: int):
        try:
            row = cust.query('customer_id == ' + str(cust_id))            
            return row.iloc[0]['segment']
        except:
            return 'Not Found!'
        
    gold = []
    silver = []
    regular = []

    for _, row in aum.iterrows():
        seg = get_segment(row['customer_id'])

        if seg == 'Not Found!':
            continue
        else:
            if seg == 'Gold':
                gold.append(row['amount'])
            elif seg == 'Silver':
                silver.append(row['amount'])
            else:
                regular.append(row['amount'])

    return pd.DataFrame(sorted(gold), columns=['amount']), pd.DataFrame(sorted(silver), columns=['amount']), pd.DataFrame(sorted(regular), columns=['amount'])

### Question 6

In [68]:
def q6(cust: pd.DataFrame, prod: pd.DataFrame):
    def get_city(cust_id: int):
        try:
            row = cust.query('customer_id == ' + str(cust_id))            
            return row.iloc[0]['province_city']
        except:
            return 'Not Found!'
        
    products = {}

    for _, row in prod.iterrows():
        cust_city = get_city(row['customer_id'])

        if cust_city == 'Not Found!':
            continue
        else:
            if products.get(cust_city) == None:
                products[cust_city] = [row['prod_ca'], row['prod_td'], row['prod_credit_card']]
            else:
                new_record = [x+y for x, y in zip(products.get(cust_city), [row['prod_ca'], row['prod_td'], row['prod_credit_card']])]
                products.update({cust_city: new_record})

    list_prod = [(city, *values) for city, values in products.items()]

    return pd.DataFrame(list_prod, columns=['province_city', 'prod_ca', 'prod_td', 'prod_credit_card'])
    
    

### Question 7

In [79]:
def q7(cust: pd.DataFrame, prod: pd.DataFrame):
    def get_segment(cust_id: int):
        try:
            row = cust.query('customer_id == ' + str(cust_id))            
            return row.iloc[0]['segment']
        except:
            return 'Not Found!'
        
    product_seg = {
        SEGMENT[0]: [0,0,0],
        SEGMENT[1]: [0,0,0],
        SEGMENT[2]: [0,0,0]
    }

    for _, row in prod.iterrows():
        seg = get_segment(row['customer_id'])

        if seg == 'Not Found!':
            continue
        else:
            new_record = [x+y for x, y in zip(product_seg.get(seg), [row['prod_ca'], row['prod_td'], row['prod_credit_card']])]
            product_seg.update({seg: new_record})

    list_ps = [(seg, *values) for seg, values in product_seg.items()]

    return pd.DataFrame(list_ps, columns=['seg', 'prod_ca', 'prod_td', 'prod_cc'])

### Question 8

In [81]:
def q8(aum: pd.DataFrame, prod: pd.DataFrame):
    def get_amount(cust_id: int):
        try:
            row = aum.query('customer_id == ' + str(cust_id))            
            return row.iloc[0]['amount']
        except:
            return 'Not Found!'
    
    amount = {
        'Credit Card': 0,
        'Others': 0
    }

    for _,row in prod.iterrows():
        if row['prod_credit_card'] == 1:
            amount.update({'Credit Card': amount.get('Credit Card')+get_amount(row['customer_id'])}) 
        else:
            amount.update({'Others': amount.get('Others')+get_amount(row['customer_id'])}) 

    return amount


## RUN IT!

In [6]:
cust_data = read_csv('cust')
aum_data = read_csv('aum')
prod_data = read_csv('prod_holding')

### Answer 1

In [20]:
a1 = q1(cust_data)

max_key = max(a1, key=a1.get)
print(max_key, a1[max_key])

regular 24174


### Answer 2

In [12]:
import itertools

a2 = q2(cust_data)

a2_sorted = dict(sorted(a2.items(), key = lambda item: item[1], reverse=True))

dict(itertools.islice(a2_sorted.items(), 5))

{'HA NOI': 3401,
 'HO CHI MINH': 3296,
 'No Info': 456,
 'HAI PHONG': 262,
 'THANH HOA': 179}

### Answer 3

In [59]:
a3 = q3(cust_data, 0)

a3_sorted = dict(sorted(a3.items(), key = lambda item: item[1], reverse=True))

print(list(a3_sorted)[0])

HO CHI MINH


### Answer 4

In [45]:
a4 = q4(aum_data, cust_data)

a4_sorted = sorted(a4.items(), key=lambda item: item[1], reverse=True)

print(list(a4_sorted)[4])

('NAM DINH', 16075956998.568098)


### Answer 5

In [56]:
def format_number(value, decimal_places=2):
    return f"{value:,.{decimal_places}f}"

gold, silver, regular = q5(aum_data, cust_data)

print('Gold: Min - Max - 25 - 50 - 75: ', 
      format_number(gold['amount'].min()), 
      format_number(gold['amount'].max()), 
      format_number(gold['amount'].quantile(0.25)), 
      format_number(gold['amount'].quantile(0.50)), 
      format_number(gold['amount'].quantile(0.75)))

print('Silver: Min - Max - 25 - 50 - 75: ', 
      format_number(silver['amount'].min()), 
      format_number(silver['amount'].max()), 
      format_number(silver['amount'].quantile(0.25)), 
      format_number(silver['amount'].quantile(0.50)), 
      format_number(silver['amount'].quantile(0.75)))

print('Regular: Min - Max - 25 - 50 - 75: ', 
      format_number(regular['amount'].min()), 
      format_number(regular['amount'].max()), 
      format_number(regular['amount'].quantile(0.25)), 
      format_number(regular['amount'].quantile(0.50)), 
      format_number(regular['amount'].quantile(0.75)))


Gold: Min - Max - 25 - 50 - 75:  0.00 63,661,883,585.00 82,458,769.14 625,000,000.00 1,441,736,462.00
Silver: Min - Max - 25 - 50 - 75:  0.00 862,318,820.80 89,911.56 1,738,411.53 25,543,670.59
Regular: Min - Max - 25 - 50 - 75:  0.00 1,500,000,002.00 23,863.38 58,642.24 228,456.15


### Answer 6

In [71]:
a6 = q6(cust_data, prod_data.fillna(0))

city_ca_idx = a6['prod_ca'].idxmax()
city_td_idx = a6['prod_td'].idxmax()
city_cc_idx = a6['prod_credit_card'].idxmax()

print('prod_ca: ', a6.iloc[city_ca_idx]['province_city'])
print('prod_td: ', a6.iloc[city_td_idx]['province_city'])
print('prod_cc: ', a6.iloc[city_cc_idx]['province_city'])

prod_ca:  HA NOI
prod_td:  HO CHI MINH
prod_cc:  HO CHI MINH


### Answer 7

In [80]:
a7 = q7(cust_data, prod_data.fillna(0))

print('Percentage of Regular, Silver and Gold seg:')
print('prod_ca:', a7.iloc[0]['prod_ca']/a7['prod_ca'].sum()*100, a7.iloc[1]['prod_ca']/a7['prod_ca'].sum()*100, a7.iloc[2]['prod_ca']/a7['prod_ca'].sum()*100)
print('prod_td:', a7.iloc[0]['prod_td']/a7['prod_td'].sum()*100, a7.iloc[1]['prod_td']/a7['prod_td'].sum()*100, a7.iloc[2]['prod_td']/a7['prod_td'].sum()*100)
print('prod_cc:', a7.iloc[0]['prod_cc']/a7['prod_cc'].sum()*100, a7.iloc[1]['prod_cc']/a7['prod_cc'].sum()*100, a7.iloc[2]['prod_cc']/a7['prod_cc'].sum()*100)

Percentage of Regular, Silver and Gold seg:
prod_ca: 3.2462479155086164 15.141745414118954 81.61200667037242
prod_td: 17.6814011676397 27.522935779816514 54.79566305254379
prod_cc: 7.422068283028203 42.65215239980208 49.92577931716971


### Answer 8

In [83]:
a8 = q8(aum_data, prod_data.dropna())

print('Credit Card:', format_number(a8.get('Credit Card')))
print('Others:', format_number(a8.get('Others')))

Credit Card: 237,831,182,187.66
Others: 405,137,904,645.93


Tâm lý e ngại khi mở thẻ tín dụng, những vụ việc liên quan đến thẻ tín dụng gần đây càng làm củng cố thêm suy nghĩ ấy!

Số đông người dùng vẫn chuộng lối sống tiết kiệm, có thể cân đối trước lượng tiền sử dụng trong tháng (?) mà không cần đến các loại thẻ trả sau?